In [1]:
import tensorflow as tf
import numpy as np
import dbGeneration as db
import random

# Data Pre-Processing

In [2]:
tensorInputs, raw_labels = db.get_tensorinputs_and_labels()
tensorInputs = np.asarray(tensorInputs)

Returned 126320 data points


### Convert labels to one-hot vector

In [3]:
pre_labels = []
for i in raw_labels:
    if i == -1:
        pre_labels.append([1,0])
    else:
        pre_labels.append([0,1])

pre_labels = np.asarray(pre_labels)

In [4]:
print(pre_labels.shape)
print(pre_labels)

(126320, 2)
[[0 1]
 [0 1]
 [0 1]
 ..., 
 [0 1]
 [0 1]
 [0 1]]


### Define training and testing data

In [5]:
print(len(tensorInputs))

126320


In [6]:
BATCH_SIZE = 64
N_SAMPLES = len(tensorInputs)
N_TRAIN = 120000
N_VALIDATION = int((N_SAMPLES - N_TRAIN)/2)
N_BATCHES = int(N_TRAIN/BATCH_SIZE)

## Shuffle data to avoid grouping among games

In [7]:
pairs = []

In [8]:
for i in range(len(tensorInputs)):
    pairs.append([tensorInputs[i], pre_labels[i]])

In [9]:
random.shuffle(pairs)

In [10]:
print(pairs[10])

[array([[ 9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.],
       [ 9., -1., -1., -1., -1., -1., -1., -1., -1.,  9.],
       [ 9., -1., -1., -1., -1., -1., -1., -1.,  1.,  9.],
       [ 9., -1., -1., -1., -1.,  1., -1.,  1.,  1.,  9.],
       [ 9., -1., -1., -1.,  1., -1.,  1., -1.,  1.,  9.],
       [ 9., -1., -1., -1., -1.,  1.,  1.,  1.,  1.,  9.],
       [ 9., -1.,  1., -1., -1., -1.,  1.,  1.,  1.,  9.],
       [ 9.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1.,  9.],
       [ 9., -1.,  1.,  1.,  1.,  1., -1., -1.,  1.,  9.],
       [ 9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.]]), array([1, 0])]


In [11]:
tensorInputs = []
pre_labels = []

for i in pairs:
    tensorInputs.append(i[0])
    pre_labels.append(i[1])

In [12]:
train_gameboards = tensorInputs[:N_TRAIN]
validation_gameboards = tensorInputs[N_TRAIN:N_TRAIN+N_VALIDATION]
test_gameboards = tensorInputs[N_TRAIN+N_VALIDATION:]

train_labels = pre_labels[:N_TRAIN]
validation_labels = pre_labels[N_TRAIN:N_TRAIN+N_VALIDATION]
test_labels = pre_labels[N_TRAIN+N_VALIDATION:]

In [13]:
print(len(train_gameboards))
print(len(validation_gameboards))
print(len(test_gameboards))

120000
3160
3160


In [14]:
print(test_gameboards[0], test_labels[0])

[[ 9.  9.  9.  9.  9.  9.  9.  9.  9.  9.]
 [ 9. -1. -1. -1. -1. -1. -1. -1. -1.  9.]
 [ 9. -1. -1. -1. -1. -1. -1. -1. -1.  9.]
 [ 9. -1. -1. -1. -1. -1. -1. -1. -1.  9.]
 [ 9. -1. -1. -1. -1. -1. -1. -1. -1.  9.]
 [ 9. -1. -1. -1. -1.  1. -1. -1. -1.  9.]
 [ 9. -1. -1. -1. -1.  1. -1. -1. -1.  9.]
 [ 9. -1. -1. -1. -1. -1. -1. -1. -1.  9.]
 [ 9. -1. -1. -1. -1. -1. -1. -1. -1.  9.]
 [ 9.  9.  9.  9.  9.  9.  9.  9.  9.  9.]] [1 0]


# Model Definition

In [15]:
gameboards = tf.placeholder(tf.float32, (None, 10, 10, 1), name="gameBoards")


In [16]:
"""w = tf.Variable(tf.truncated_normal([64, 2], mean=1.0, stddev=0.5), name="weights")
b = tf.Variable(tf.zeros([1,2]), name="bias")"""

'w = tf.Variable(tf.truncated_normal([64, 2], mean=1.0, stddev=0.5), name="weights")\nb = tf.Variable(tf.zeros([1,2]), name="bias")'

In [17]:

# First Convulutional layer, small 2x2 filter
conv1 = tf.layers.conv2d(gameboards, 32, 2, padding="same", name="Conv1", activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(conv1, 2, 2, name="Pool1")

# Increased the second convolution size to 4x4, larger frames aiming to capture larger features.
conv2 = tf.layers.conv2d(pool1, 64, 4, padding="same", name="Conv2", activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(conv2, 2, 2, name="Pool2")

# Reshape the 2D tensor back to 1D to be fed into "Dense"
# Flatten out the pooling - GET THIS NUMBER FROM POOL.SHAPE
pool2_flat = tf.reshape(pool2, (-1, int(2*2*64)), name="Pool2_Flat")


# The dropout allows us to train a subset of the neurons at any given iteration.  
keep_prob = tf.placeholder(tf.float32, name="Keep_Probability")


# A dense layer with dropout
# DENSE - a fully connected linear transofmration of every dimension of the data
dense = tf.layers.dense(pool2_flat, int(2*2*64), activation=tf.nn.relu, name="Dense")

# DROPOUT - if set to 0.5, rendomly select 50% of the neurons to ignore (different with each computation)
dropout = tf.nn.dropout(dense, keep_prob, name="Dropout")

dense2 = tf.layers.dense(dropout, int(2*64), activation=tf.nn.relu, name="Dense2")
dropout2 = tf.nn.dropout(dense2, keep_prob, name="Dropout2")

# A dense layer to classify the final values. Only 2 neurons. 
predictions = tf.layers.dense(dropout2, 2, activation=None, name="Predictions")

In [18]:
pool2.shape

TensorShape([Dimension(None), Dimension(2), Dimension(2), Dimension(64)])

In [10]:
# Add two convolution layers with max pooling
#conv1 = tf.layers.conv1d(gameboards_1d, 32, 5, padding="same", name="Conv1")

## 2D

conv1 = tf.layers.conv2d(gameboards_2d, 64, 5, padding="same", name="Conv1")
pool1 = tf.layers.max_pooling2d(conv1, 4, 4, name="Pool1")
conv2 = tf.layers.conv2d(pool1, 128, 5, padding="same", name="Conv2")
pool2 = tf.layers.max_pooling2d(conv2, 2, 2, name="Pool2")

# Reshape the 2D tensor back to 1D to be fed into "Dense"
pool2_flat = tf.reshape(pool2, (-1, 8*8*2), name="Pool2_Flat")
'''

## 1D
conv1 = tf.layers.conv1d(gameboards_1d, 32, 5, padding="same", name="Conv1")
pool1 = tf.layers.max_pooling1d(conv1, 2, 2, name="Pool1")
conv2 = tf.layers.conv1d(pool1, 200, 5, padding="same", name="Conv2")
pool2 = tf.layers.max_pooling1d(conv2, 2, 2, name="Pool2")

# 50 is batch size, image is 8*8 (?)
pool2_flat = tf.reshape(pool2, (-1, 8*8*50), name="Pool2_Flat")
'''

# A dense layer with dropout
dense = tf.layers.dense(pool2_flat, 512, activation=tf.nn.relu, name="Dense")
keep_prob = tf.placeholder(tf.float32, name="Keep_Probability")
dropout = tf.nn.dropout(dense, keep_prob, name="Dropout")

# The original dense layer to compute logits that are later used for classification
logits = tf.layers.dense(dropout, 2, activation=None, name="Logits")

In [11]:
# We can take a look at the shapes of some tensors
(pool2.shape, pool2_flat.shape, dropout.shape, logits.shape)

(TensorShape([Dimension(None), Dimension(1), Dimension(1), Dimension(128)]),
 TensorShape([Dimension(None), Dimension(128)]),
 TensorShape([Dimension(None), Dimension(512)]),
 TensorShape([Dimension(None), Dimension(2)]))

In [12]:
#logits = tf.add(tf.matmul(gameboards, w), b)

In [13]:
w.shape, b.shape, logits.shape

(TensorShape([Dimension(64), Dimension(2)]),
 TensorShape([Dimension(1), Dimension(2)]),
 TensorShape([Dimension(None), Dimension(2)]))

In [14]:
# None, as in, this is not yet defined, there could be any number of them input. 
# 2, as in, there are two elements in the one-hot vector

labels = tf.placeholder(tf.int32, [None, 2], name="labels")

In [15]:
# This loss is the elementwise loss
#loss = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)

# We want the mean loss
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits))

In [16]:
loss.shape

TensorShape([])

In [17]:
train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

## Define an error rate to evaluate model performance


In [18]:
with tf.name_scope("Error"):
    error = tf.reduce_mean(
        tf.cast(
            tf.not_equal(tf.argmax(logits, axis=1), tf.argmax(labels, axis=1)), tf.float32))

In [19]:
tf.summary.scalar("Loss", loss)
tf.summary.scalar("Error", error)

<tf.Tensor 'Error_1:0' shape=() dtype=string>

In [20]:
number_of_training_samples = len(list(train_gameboards))
print(number_of_training_samples)

number_of_samples_per_batch = 64

number_of_batches = int(number_of_training_samples / number_of_samples_per_batch)
print(number_of_batches)

14000
218


In [21]:
sess = tf.Session()

In [22]:
sess.run(tf.global_variables_initializer())

In [23]:
print(number_of_batches)

218


In [24]:
print(number_of_samples_per_batch)

64


In [25]:
#for epoch in range(5):
    #randomise the data

for i in range(number_of_batches):
    first_index = i*number_of_samples_per_batch
    second_index = (i*number_of_samples_per_batch) + number_of_samples_per_batch
    batch = train_gameboards[first_index:second_index]
    batchLabels = train_labels[first_index:second_index]
    
    if i % 50 == 0:
        Error, Loss = sess.run([error, loss], feed_dict={gameboards:batch, labels:batchLabels, keep_prob:1.0})
        print("Loss: {}, \tError: {}".format(Loss, Error))
    sess.run(train, feed_dict={gameboards:batch, labels:batchLabels, keep_prob:1.0})


Loss: 0.693146288394928, 	Error: 0.484375
Loss: 0.6975248456001282, 	Error: 0.515625
Loss: 0.690526008605957, 	Error: 0.5
Loss: 0.6898193359375, 	Error: 0.46875
Loss: 0.6749790906906128, 	Error: 0.390625


In [26]:
Error = sess.run(error, feed_dict={gameboards:test_gameboards, labels:test_labels, keep_prob:1.0})

In [27]:
print(Error)

0.47486
